In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col,lit
from pyspark.sql.functions import hour, weekofyear, date_format,dayofweek
from pyspark.sql.functions import year,month, dayofmonth
from pyspark.sql.functions import monotonically_increasing_id
#from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from pyspark.sql.types import IntegerType, DateType

In [2]:
pd.set_option('display.max_colwidth', -1)
#from pyspark import SparkContext
#sc = SparkContext()


In [3]:

def create_spark_session():
    """Creates SparkSession. General configurarion
    of the script.
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [4]:
spark=create_spark_session()
input_data = "/home/gari/Desktop/final_project/input_data/arxiv-metadata-oai-snapshot.json"
output_data="/home/gari/Desktop/final_project/parquet_area/"

In [5]:
df = spark.read.json(input_data)

In [6]:
df=df.limit(5000)

In [7]:
df.limit(50).toPandas()

,abstract,authors,authors_parsed,categories,comments,doi,id,journal-ref,license,report-no,submitter,title,update_date,versions
0,"A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","[[Balázs, C., ], [Berger, E. L., ], [Nadolsky, P. M., ], [Yuan, C. -P., ]]",hep-ph,"37 pages, 15 figures; published version",10.1103/PhysRevD.76.013009,0704.0001,"Phys.Rev.D76:013009,2007",None,ANL-HEP-PR-07-12,Pavel Nadolsky,Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies,2008-11-26,"[(Mon, 2 Apr 2007 19:18:42 GMT, v1), (Tue, 24 Jul 2007 20:10:27 GMT, v2)]"
1,"We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n",Ileana Streinu and Louis Theran,"[[Streinu, Ileana, ], [Theran, Louis, ]]",math.CO cs.CG,To appear in Graphs and Combinatorics,None,0704.0002,None,http://arxiv.org/licenses/nonexclusive-distrib/1.0/,None,Louis Theran,Sparsity-certifying Graph Decompositions,2008-12-13,"[(Sat, 31 Mar 2007 02:26:18 GMT, v1), (Sat, 13 Dec 2008 17:26:00 GMT, v2)]"
2,"The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n",Hongjun Pan,"[[Pan, Hongjun, ]]",physics.gen-ph,"23 pages, 3 figures",None,0704.0003,None,None,None,Hongjun Pan,The evolution of the Earth-Moon system based on the dark matter field\n fluid model,2008-01-13,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]"
3,We show that a determinant of Stirling cy

In [13]:
df.select('versions').printSchema()

root
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [17]:
df.selectExpr("versions","explode(versions)").show(2)

+--------------------+--------------------+
|            versions|                 col|
+--------------------+--------------------+
|[[Mon, 2 Apr 2007...|[Mon, 2 Apr 2007 ...|
|[[Mon, 2 Apr 2007...|[Tue, 24 Jul 2007...|
+--------------------+--------------------+
only showing top 2 rows



In [22]:
df.selectExpr("versions").limit(10).toPandas()

,versions
0,"[(Mon, 2 Apr 2007 19:18:42 GMT, v1), (Tue, 24 Jul 2007 20:10:27 GMT, v2)]"
1,"[(Sat, 31 Mar 2007 02:26:18 GMT, v1), (Sat, 13 Dec 2008 17:26:00 GMT, v2)]"
2,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]"
3,"[(Sat, 31 Mar 2007 03:16:14 GMT, v1)]"
4,"[(Mon, 2 Apr 2007 18:09:58 GMT, v1)]"
5,"[(Sat, 31 Mar 2007 04:24:59 GMT, v1)]"
6,"[(Sat, 31 Mar 2007 04:27:22 GMT, v1), (Wed, 22 Aug 2007 22:42:11 GMT, v2)]"
7,"[(Sat, 31 Mar 2007 04:47:20 GMT, v1), (Thu, 10 Apr 2008 08:42:28 GMT, v2), (Tue, 1 Jul 2008 18:54:28 GMT, v3)]"
8,"[(Mon, 2 Apr 2007 19:41:34 GMT, v1)]"
9,"[(Sat, 31 Mar 2007 05:10:16 GMT, v1)]"


In [23]:
df.selectExpr("versions","explode(versions)").limit(10).toPandas()

,versions,col
0,"[(Mon, 2 Apr 2007 19:18:42 GMT, v1), (Tue, 24 Jul 2007 20:10:27 GMT, v2)]","(Mon, 2 Apr 2007 19:18:42 GMT, v1)"
1,"[(Mon, 2 Apr 2007 19:18:42 GMT, v1), (Tue, 24 Jul 2007 20:10:27 GMT, v2)]","(Tue, 24 Jul 2007 20:10:27 GMT, v2)"
2,"[(Sat, 31 Mar 2007 02:26:18 GMT, v1), (Sat, 13 Dec 2008 17:26:00 GMT, v2)]","(Sat, 31 Mar 2007 02:26:18 GMT, v1)"
3,"[(Sat, 31 Mar 2007 02:26:18 GMT, v1), (Sat, 13 Dec 2008 17:26:00 GMT, v2)]","(Sat, 13 Dec 2008 17:26:00 GMT, v2)"
4,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]","(Sun, 1 Apr 2007 20:46:54 GMT, v1)"
5,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]","(Sat, 8 Dec 2007 23:47:24 GMT, v2)"
6,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]","(Sun, 13 Jan 2008 00:36:28 GMT, v3)"
7,"[(Sat, 31 Mar 2007 03:16:14 GMT, v1)]","(Sat, 31 Mar 2007 03:16:14 GMT, v1)"
8,"[(Mon, 2 Apr 2007 18:09:58 GMT, v1)]","(Mon, 2 Apr 2007 18:09:58 GMT, v1)"
9,"[(Sat, 31 Mar 2007 04:24:59 GMT, v1)]","(Sat, 31 Mar 2007 04:24:59 GMT, v1)"


In [35]:
import datetime
estDF.select("start_date", date_format("start_date", 'dd/MM/yyyy').alias("dt_format")).show()

In [45]:
get_hour = udf(lambda x: dayofmonth(x))

In [49]:
test_df = df_versions.withColumn("time", dayofmonth((df_versions.created)))

In [51]:
df_versions.select("created", date_format("created", 'dd/MM/yyyy').alias("dt_format")).show()

+--------------------+---------+
|             created|dt_format|
+--------------------+---------+
|Mon, 2 Apr 2007 1...|     null|
|Tue, 24 Jul 2007 ...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Sat, 13 Dec 2008 ...|     null|
|Sun, 1 Apr 2007 2...|     null|
|Sat, 8 Dec 2007 2...|     null|
|Sun, 13 Jan 2008 ...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Mon, 2 Apr 2007 1...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Wed, 22 Aug 2007 ...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Thu, 10 Apr 2008 ...|     null|
|Tue, 1 Jul 2008 1...|     null|
|Mon, 2 Apr 2007 1...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Sat, 31 Mar 2007 ...|     null|
|Tue, 19 Aug 2008 ...|     null|
|Wed, 20 Aug 2008 ...|     null|
+--------------------+---------+
only showing top 20 rows



In [53]:
from datetime import datetime
import pytz
from pyspark.sql.functions import udf, to_date, to_utc_timestamp

## Converting date string format
def getDate(x):
    if x is not None:
        return str(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

## UDF declaration
date_fn = udf(getDate, StringType())

## Converting datatype in spark dataframe
df_versions = df_versions.withColumn("created", to_utc_timestamp(date_fn("created"),"UTC")) 

In [66]:
from pyspark.sql.functions import to_timestamp, date_format,from_unixtime,unix_timestamp

In [ ]:
df_versions = df_versions.withColumn("date", to_utc_timestamp(date_fn("created"),"UTC")) 

In [76]:
df_versions=df.selectExpr("id","versions","explode(versions) as info_versions")
df_versions=df_versions.withColumn("created", col("info_versions.created"))
df_versions=df_versions.withColumn("version", col("info_versions.version"))
df_versions=df_versions.selectExpr("id","created","version")
df_versions.show(3)

+---------+--------------------+-------+
|       id|             created|version|
+---------+--------------------+-------+
|0704.0001|Mon, 2 Apr 2007 1...|     v1|
|0704.0001|Tue, 24 Jul 2007 ...|     v2|
|0704.0002|Sat, 31 Mar 2007 ...|     v1|
+---------+--------------------+-------+
only showing top 3 rows



In [13]:
df_abstracts=df.select("id","abstract")
df_abstracts.show(2)

+---------+--------------------+
|       id|            abstract|
+---------+--------------------+
|0704.0001|  A fully differe...|
|0704.0002|  We describe a n...|
+---------+--------------------+
only showing top 2 rows



In [15]:
df_abstracts.describe()

DataFrame[summary: string, id: string, abstract: string]

In [25]:
df_categories=df.selectExpr("id","explode(split(categories,' ')) as categories")

In [27]:
df_categories.show()

+---------+-----------------+
|       id|       categories|
+---------+-----------------+
|0704.0001|           hep-ph|
|0704.0002|          math.CO|
|0704.0002|            cs.CG|
|0704.0003|   physics.gen-ph|
|0704.0004|          math.CO|
|0704.0005|          math.CA|
|0704.0005|          math.FA|
|0704.0006|cond-mat.mes-hall|
|0704.0007|            gr-qc|
|0704.0008|cond-mat.mtrl-sci|
|0704.0009|         astro-ph|
|0704.0010|          math.CO|
|0704.0011|          math.NT|
|0704.0011|          math.AG|
|0704.0012|          math.NT|
|0704.0013|          math.NT|
|0704.0014|          math.CA|
|0704.0014|          math.AT|
|0704.0015|           hep-th|
|0704.0016|           hep-ph|
+---------+-----------------+
only showing top 20 rows



In [20]:
df_categories.count()

5000

In [24]:
#split("letters", ", ")

df.selectExpr("id","explode(split(categories,' ')) as e").show()

+---------+-----------------+
|       id|                e|
+---------+-----------------+
|0704.0001|           hep-ph|
|0704.0002|          math.CO|
|0704.0002|            cs.CG|
|0704.0003|   physics.gen-ph|
|0704.0004|          math.CO|
|0704.0005|          math.CA|
|0704.0005|          math.FA|
|0704.0006|cond-mat.mes-hall|
|0704.0007|            gr-qc|
|0704.0008|cond-mat.mtrl-sci|
|0704.0009|         astro-ph|
|0704.0010|          math.CO|
|0704.0011|          math.NT|
|0704.0011|          math.AG|
|0704.0012|          math.NT|
|0704.0013|          math.NT|
|0704.0014|          math.CA|
|0704.0014|          math.AT|
|0704.0015|           hep-th|
|0704.0016|           hep-ph|
+---------+-----------------+
only showing top 20 rows



In [64]:
df_versions.withColumn("dd",from_unixtime(col("created"), format='yyyy-MM-dd HH:mm:ss')).show(10)

, regexp_extract(col('Notes'), '(.)(by)(\s+)(\w+)', 4)

+---------+--------------------+-------+----+
|       id|             created|version|  dd|
+---------+--------------------+-------+----+
|0704.0001|Mon, 2 Apr 2007 1...|     v1|null|
|0704.0001|Tue, 24 Jul 2007 ...|     v2|null|
|0704.0002|Sat, 31 Mar 2007 ...|     v1|null|
|0704.0002|Sat, 13 Dec 2008 ...|     v2|null|
|0704.0003|Sun, 1 Apr 2007 2...|     v1|null|
|0704.0003|Sat, 8 Dec 2007 2...|     v2|null|
|0704.0003|Sun, 13 Jan 2008 ...|     v3|null|
|0704.0004|Sat, 31 Mar 2007 ...|     v1|null|
|0704.0005|Mon, 2 Apr 2007 1...|     v1|null|
|0704.0006|Sat, 31 Mar 2007 ...|     v1|null|
+---------+--------------------+-------+----+
only showing top 10 rows



In [80]:
from pyspark.sql.functions import regexp_extract

In [96]:
df_versions=df_versions.withColumn("mouth", regexp_extract(col('created'), '(,)(\s+)(\w+)(\s+)(\w+)', 5))
df_versions=df_versions.withColumn("year", regexp_extract(col('created'), '(,)(\s+)(\w+)(\s+)(\w+)(\s+)(\w+)', 7))
df_versions.show(10)

+---------+--------------------+-------+-----+----+
|       id|             created|version|mouth|year|
+---------+--------------------+-------+-----+----+
|0704.0001|Mon, 2 Apr 2007 1...|     v1|  Apr|2007|
|0704.0001|Tue, 24 Jul 2007 ...|     v2|  Jul|2007|
|0704.0002|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
|0704.0002|Sat, 13 Dec 2008 ...|     v2|  Dec|2008|
|0704.0003|Sun, 1 Apr 2007 2...|     v1|  Apr|2007|
|0704.0003|Sat, 8 Dec 2007 2...|     v2|  Dec|2007|
|0704.0003|Sun, 13 Jan 2008 ...|     v3|  Jan|2008|
|0704.0004|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
|0704.0005|Mon, 2 Apr 2007 1...|     v1|  Apr|2007|
|0704.0006|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
+---------+--------------------+-------+-----+----+
only showing top 10 rows



In [71]:


df_versions.withColumn("dd", unix_timestamp(df_versions.created,'zzz, dd MMM yyyy HH:mm:ss zzz')   ).show(10)

+---------+--------------------+-------+----------+
|       id|             created|version|        dd|
+---------+--------------------+-------+----------+
|0704.0001|Mon, 2 Apr 2007 1...|     v1|      null|
|0704.0001|Tue, 24 Jul 2007 ...|     v2|      null|
|0704.0002|Sat, 31 Mar 2007 ...|     v1|1175307978|
|0704.0002|Sat, 13 Dec 2008 ...|     v2|1229189160|
|0704.0003|Sun, 1 Apr 2007 2...|     v1|      null|
|0704.0003|Sat, 8 Dec 2007 2...|     v2|      null|
|0704.0003|Sun, 13 Jan 2008 ...|     v3|      null|
|0704.0004|Sat, 31 Mar 2007 ...|     v1|1175310974|
|0704.0005|Mon, 2 Apr 2007 1...|     v1|      null|
|0704.0006|Sat, 31 Mar 2007 ...|     v1|1175315099|
+---------+--------------------+-------+----------+
only showing top 10 rows



In [98]:
from pyspark.sql.functions import col, concat_ws
df_author=df.selectExpr("id","explode(authors_parsed) as e")
df_author=df_author.withColumn("author",concat_ws(" ",col("e")))
df_author=df_author.select("id","author")

In [99]:
df_author.limit(10).toPandas()

,id,author
0,0704.0001,Balázs C.
1,0704.0001,Berger E. L.
2,0704.0001,Nadolsky P. M.
3,0704.0001,Yuan C. -P.
4,0704.0002,Streinu Ileana
5,0704.0002,Theran Louis
6,0704.0003,Pan Hongjun
7,0704.0004,Callan David
8,0704.0005,Abu-Shammala Wael
9,0704.0005,Torchinsky Alberto


In [170]:

df2 = spark.read.csv('/home/gari/Desktop/final_project/input_data/papers.csv',header=True)

In [173]:
df2.na.drop().show(10)

+--------------------+------------+--------------------+-----------------+--------------+--------------------+--------------------+
|                  id|        year|               title|       event_type|      pdf_name|            abstract|          paper_text|
+--------------------+------------+--------------------+-----------------+--------------+--------------------+--------------------+
|         Furthermore|  add (x"" y|) to S;-l to prod...|             i.e.|             S|                 = S|          _l U {(x""|
|       [10] Bachmann|       C. M.|              Cooper|             L. N|         Dembo|       A. & Zeitouni|           O. (to be|
|             Fregnac|          Y.|               Shulz|               D.|        Thorpe|  S. and Bienenstock| E. A cellular an...|
|        II p(y; I x;|         DCn|                  j)| Mt}j II p(Y; Ix;|           DCn|                  j)|                 M2)|
|[Ford et al. 89] ...|         I. |        Titterington|             D.M.|  

In [176]:
df2.na.drop().count()

23701

In [177]:
df2.na.drop().limit(10).toPandas()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,Furthermore,"add (x"""" y",) to S;-l to produce Sa,i.e.,S,= S,"_l U {(x"""""
1,[10] Bachmann,C. M.,Cooper,L. N,Dembo,A. & Zeitouni,O. (to be
2,Fregnac,Y.,Shulz,D.,Thorpe,S. and Bienenstock,E. A cellular analogue of visual cortical plasticity. Nature (Lond.) 333
3,II p(y; I x;,DCn,j),Mt}j II p(Y; Ix;,DCn,j),M2)
4,[Ford et al. 89] Ford,I.,Titterington,D.M.,Kitsos,C.P. (1989): Recent Advances in Nonlinear Design. Technometrics,31
5,i.e.,translation,rotation,dilation,elongation,and shear,do not change the underlying shape of an object so we want the deformation energy to be invariant under
6,for hand-printed character recognition. In Moody,J. E.,Hanson,S. J.,and Lippmann,R. P.,editors
7,Jacobs,R. A.,Jordan,M. 1.,Nowlan,S. J.,and Hinton
8,characterize objects. In Lippmann,R. P.,Moody,J. E.,and Touretzky,D. S.,editors
9,Durbin,R. M.,Miall,c.,and Mitchison,G. J.,editors


In [182]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect('/home/gari/Desktop/final_project/input_data/database.sqlite')

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute("pragma table_info('papers')"):
    print(row)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'year', 'INTEGER', 0, None, 0)
(2, 'title', 'TEXT', 0, None, 0)
(3, 'event_type', 'TEXT', 0, None, 0)
(4, 'pdf_name', 'TEXT', 0, None, 0)
(5, 'abstract', 'TEXT', 0, None, 0)
(6, 'paper_text', 'TEXT', 0, None, 0)


In [187]:
for row in cur.execute("select count(*) from papers where abstract != 'Abstract Missing' "):
    print(row)

(3924,)


In [188]:

for row in cur.execute("pragma table_info('paper_authors')"):
    print(row)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'paper_id', 'INTEGER', 0, None, 0)
(2, 'author_id', 'INTEGER', 0, None, 0)


In [189]:
df3 = sqlContext.read.format('jdbc').\
     options(url='jdbc:sqlite:Chinook_Sqlite.sqlite',\
     dbtable='/home/gari/Desktop/final_project/input_data/database.sqlite',driver='org.sqlite.JDBC').load()

NameError: name 'sqlContext' is not defined

In [37]:
import sqlite3
import pandas as pd

db_path = "/home/gari/Desktop/final_project/input_data/database.sqlite"
query = "select id,year ,title,abstract  from papers where abstract != 'Abstract Missing' "

conn = sqlite3.connect(db_path)
a_pandas_df = pd.read_sql_query(query, conn)



#a_spark_df = spark.createDataFrame(a_pandas_df)

In [38]:
a_pandas_df.to_csv('/home/gari/Desktop/final_project/input_data/NIPS.csv',sep='|',index=False)

In [61]:
a_pandas_df

,id,year,title,abstract
0,1861,2000,Algorithms for Non-negative Matrix Factorization,"Non-negative matrix factorization (NMF) has previously been shown to \r\nbe a useful decomposition for multivariate data. Two different multi- \r\nplicative algorithms for NMF are analyzed. They differ only slightly in \r\nthe multiplicative factor used in the update rules. One algorithm can be \r\nshown to minimize the conventional least squares error while the other \r\nminimizes the generalized Kullback-Leibler divergence. The monotonic \r\nconvergence of both algorithms can be proven using an auxiliary func- \r\ntion analogous to that used for proving convergence of the Expectation- \r\nMaximization algorithm. The algorithms can also be interpreted as diag- \r\nonally rescaled gradient descent, where the rescaling factor is optimally \r\nchosen to ensure convergence."
1,1975,2001,Characterizing Neural Gain Control using Spike-triggered Covariance,"Spike-triggered averaging techniques are effective for linear characterization of neural responses. But neurons exhibit important nonlinear behaviors, such as gain control, that are not captured by such analyses. We describe a spike-triggered covariance method for retrieving suppressive components of the gain control signal in a neuron. We demonstrate the method in simulation and on retinal ganglion cell data. Analysis of physiological data reveals significant suppressive axes and explains neural nonlinearities. This method should be applicable to other sensory areas and modalities."
2,3163,2007,Competition Adds Complexity,"It is known that determinining whether a DEC-POMDP, namely, a cooperative partially observable stochastic game (POSG), has a cooperative strategy with positive expected reward is complete for NEXP. It was not known until now how cooperation affected that complexity. We show that, for competitive POSGs, the complexity of determining whether one team has a positive-expected-reward strategy is complete for the class NEXP with an oracle for NP."
3,3164,2007,Efficient Principled Learning of Thin Junction Trees,"We present the first truly polynomial algorithm for learning the structure of bounded-treewidth junction trees -- an attractive subclass of probabilistic graphical models that permits both the compact representation of probability distributions and efficient exact inference. For a constant treewidth, our algorithm has polynomial time and sample complexity, and provides strong theoretical guarantees in terms of $KL$ divergence from the true distribution. We also present a lazy extension of our approach that leads to very significant speed ups in practice, and demonstrate the viability of our method empirically, on several real world datasets. One of our key new theoretical insights is a method for bounding the conditional mutual information of arbitrarily large sets of random variables with only a polynomial number of mutual information computations on fixed-size subsets of variables, when the underlying distribution can be approximated by a bounded treewidth junction tree."
4,3167,2007,Regularized Boost for Semi-Supervised Learning,"Semi-supervised inductive learning concerns how to learn a decision rule from a data set containing both labeled and unlabeled data. Several boosting algorithms have been extended to semi-supervised learning with various strategies. To our knowledge, however, none of them takes local smoothness constraints among data into account during ensemble learning. In this paper, we introduce a local smoothness regularizer to semi-supervised boosting algorithms based on the universal optimization framework of margin cost functionals. Our regularizer is applicable to existing semi-supervised boosting algorithms to improve their generalization and speed up their training. Comparative results on synthetic, benchmark and real world tasks demonstrate the effectiveness of our local smoothness regularizer. We discuss relevant issues and relate our regularizer to

In [56]:
df_abstract_nips=spark.read.format("csv").option("delimiter", "|").option("header", "True").load("/home/gari/Desktop/final_project/input_data/NIPS.csv")
df_abstract_nips=df_abstract_nips.withColumn("origin",lit("NIPS"))

In [57]:
df_abstracts=df.select("id","abstract").withColumn("origin",lit("arXiv")).union(df_abstract_nips.select("id","abstract","origin"))
#df_abstracts.write.mode("overwrite").parquet(output_data+"abstracts")

In [58]:
df_abstracts.show(10)

+---------+--------------------+------+
|       id|            abstract|origin|
+---------+--------------------+------+
|0704.0001|  A fully differe...| arXiv|
|0704.0002|  We describe a n...| arXiv|
|0704.0003|  The evolution o...| arXiv|
|0704.0004|  We show that a ...| arXiv|
|0704.0005|  In this paper w...| arXiv|
|0704.0006|  We study the tw...| arXiv|
|0704.0007|  A rather non-st...| arXiv|
|0704.0008|  A general formu...| arXiv|
|0704.0009|  We discuss the ...| arXiv|
|0704.0010|  Partial cubes a...| arXiv|
+---------+--------------------+------+
only showing top 10 rows



In [59]:
df_abstracts.count()

1756988

In [60]:
df_abstract_nips.show(10)

+----+----+--------------------+--------------------+------+
|  id|year|               title|            abstract|origin|
+----+----+--------------------+--------------------+------+
|1861|2000|Algorithms for No...| Non-negative mat...|  NIPS|
|1975|2001|Characterizing Ne...|Spike-triggered a...|  NIPS|
|3163|2007|Competition Adds ...|It is known that ...|  NIPS|
|3164|2007|Efficient Princip...|We present the fi...|  NIPS|
|3167|2007|Regularized Boost...|Semi-supervised i...|  NIPS|
|3168|2007|Simplified Rules ...|We show that unde...|  NIPS|
|3169|2007|Predicting human ...|Under natural vie...|  NIPS|
|3171|2007|Mining Internet-S...|Large repositorie...|  NIPS|
|3172|2007|Continuous Time P...|We construct a bi...|  NIPS|
|3174|2007|An online Hebbian...|Independent compo...|  NIPS|
+----+----+--------------------+--------------------+------+
only showing top 10 rows



In [62]:
df_papers_nips = spark.read.format("csv").option("delimiter", 
            "|").option("header", 
            "True").load(
                "/home/gari/Desktop/final_project/input_data/NIPS.csv")

In [63]:
df_papers_nips.show(10)

+----+----+--------------------+--------------------+
|  id|year|               title|            abstract|
+----+----+--------------------+--------------------+
|1861|2000|Algorithms for No...| Non-negative mat...|
|1975|2001|Characterizing Ne...|Spike-triggered a...|
|3163|2007|Competition Adds ...|It is known that ...|
|3164|2007|Efficient Princip...|We present the fi...|
|3167|2007|Regularized Boost...|Semi-supervised i...|
|3168|2007|Simplified Rules ...|We show that unde...|
|3169|2007|Predicting human ...|Under natural vie...|
|3171|2007|Mining Internet-S...|Large repositorie...|
|3172|2007|Continuous Time P...|We construct a bi...|
|3174|2007|An online Hebbian...|Independent compo...|
+----+----+--------------------+--------------------+
only showing top 10 rows



In [65]:
from pyspark.sql.functions import regexp_extract
df_versions=df.selectExpr("id","versions","explode(versions) as info_versions")
df_versions=df_versions.withColumn("created", col("info_versions.created"))
df_versions=df_versions.withColumn("version", col("info_versions.version"))
df_versions=df_versions.selectExpr("id","created","version")
df_versions=df_versions.withColumn("mouth", regexp_extract(col('created'), '(,)(\s+)(\w+)(\s+)(\w+)', 5))
df_versions=df_versions.withColumn("year", regexp_extract(col('created'), '(,)(\s+)(\w+)(\s+)(\w+)(\s+)(\w+)', 7))


In [66]:
df_versions.show(10)

+---------+--------------------+-------+-----+----+
|       id|             created|version|mouth|year|
+---------+--------------------+-------+-----+----+
|0704.0001|Mon, 2 Apr 2007 1...|     v1|  Apr|2007|
|0704.0001|Tue, 24 Jul 2007 ...|     v2|  Jul|2007|
|0704.0002|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
|0704.0002|Sat, 13 Dec 2008 ...|     v2|  Dec|2008|
|0704.0003|Sun, 1 Apr 2007 2...|     v1|  Apr|2007|
|0704.0003|Sat, 8 Dec 2007 2...|     v2|  Dec|2007|
|0704.0003|Sun, 13 Jan 2008 ...|     v3|  Jan|2008|
|0704.0004|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
|0704.0005|Mon, 2 Apr 2007 1...|     v1|  Apr|2007|
|0704.0006|Sat, 31 Mar 2007 ...|     v1|  Mar|2007|
+---------+--------------------+-------+-----+----+
only showing top 10 rows



In [81]:
df_versions_nips=df_papers_nips.select('id','year').\
                    withColumn('created',lit('no info')).\
                    withColumn('version',lit('no info')).\
                    withColumn('mouth',lit('no info'))
df_versions_nips=df_versions_nips.select('id','created','version','mouth','year')

In [83]:
df_versions.union(df_versions_nips).where(col('id')=='1861').show(10)

+----+-------+-------+-------+----+
|  id|created|version|  mouth|year|
+----+-------+-------+-------+----+
|1861|no info|no info|no info|2000|
+----+-------+-------+-------+----+



In [82]:
df_versions_nips.show(10)

+----+-------+-------+-------+----+
|  id|created|version|  mouth|year|
+----+-------+-------+-------+----+
|1861|no info|no info|no info|2000|
|1975|no info|no info|no info|2001|
|3163|no info|no info|no info|2007|
|3164|no info|no info|no info|2007|
|3167|no info|no info|no info|2007|
|3168|no info|no info|no info|2007|
|3169|no info|no info|no info|2007|
|3171|no info|no info|no info|2007|
|3172|no info|no info|no info|2007|
|3174|no info|no info|no info|2007|
+----+-------+-------+-------+----+
only showing top 10 rows



In [15]:
import re

#Compilamos símbolos a luego serán removidos de nuestro texto
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

#Compilamos símbolos que no sean numeros ni letras, ni vocales con acento.
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')


def text_prepare(text):
    """
        text: a string
        return: standarized initial string
    """
    text = text.lower() # Todo a minuscula
    text = REPLACE_BY_SPACE_RE.sub(" ",text) # Reemplazamos REPLACE_BY_SPACE_RE symboos por espacio
    text = BAD_SYMBOLS_RE.sub(" ",text) # borramos los BAD_SYMBOLS_RE
    text = re.sub(r'\s+'," ",text) # Eliminamos los espacios en blanco repetidos
    return None



In [16]:
clean_text = udf(lambda x: text_prepare(x))

SyntaxError: invalid syntax (<ipython-input-49-81ba71e30b1f>, line 5)

In [14]:
! x

/usr/bin/python3


In [16]:
! echo $PYSPARK_DRIVER_PYTHON

In [19]:
!export PYSPARK_PYTHON=/home/gari/anaconda3/envs/DE/bin/python

In [8]:
! echo $PYSPARK_PYTHON

/home/gari/anaconda3/envs/DE/bin/python


In [27]:
df.na.

AttributeError: 'DataFrameNaFunctions' object has no attribute 'show'

In [36]:
df_null.na.drop()

DataFrame[abstract: string, authors: string, authors_parsed: array<array<string>>, categories: string, comments: string, doi: string, id: string, journal-ref: string, license: string, report-no: string, submitter: string, title: string, update_date: string, versions: array<struct<created:string,version:string>>, processed_abstrac: string]

In [23]:
df2

DataFrame[abstract: string, authors: string, authors_parsed: array<array<string>>, categories: string, comments: string, doi: string, id: string, journal-ref: string, license: string, report-no: string, submitter: string, title: string, update_date: string, versions: array<struct<created:string,version:string>>]

In [40]:
df_null.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|         submitter|               title|update_date|            versions|processed_abstrac|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001

In [41]:
dffff=df_null.na.drop()

In [42]:
dffff.show(10)

+--------+-------+--------------+----------+--------+---+---+-----------+-------+---------+---------+-----+-----------+--------+-----------------+
|abstract|authors|authors_parsed|categories|comments|doi| id|journal-ref|license|report-no|submitter|title|update_date|versions|processed_abstrac|
+--------+-------+--------------+----------+--------+---+---+-----------+-------+---------+---------+-----+-----------+--------+-----------------+
+--------+-------+--------------+----------+--------+---+---+-----------+-------+---------+---------+-----+-----------+--------+-----------------+



In [43]:
df_null.show(20)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------------+
|            abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|         submitter|               title|update_date|            versions|processed_abstrac|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+-----------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001

In [44]:
df.limit(10).toPD

AttributeError: 'DataFrame' object has no attribute 'toPD'

In [45]:
df_pandas=df.limit(10).toPandas()

In [46]:
df_pandas

,abstract,authors,authors_parsed,categories,comments,doi,id,journal-ref,license,report-no,submitter,title,update_date,versions
0,"A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","[[Balázs, C., ], [Berger, E. L., ], [Nadolsky, P. M., ], [Yuan, C. -P., ]]",hep-ph,"37 pages, 15 figures; published version",10.1103/PhysRevD.76.013009,0704.0001,"Phys.Rev.D76:013009,2007",None,ANL-HEP-PR-07-12,Pavel Nadolsky,Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies,2008-11-26,"[(Mon, 2 Apr 2007 19:18:42 GMT, v1), (Tue, 24 Jul 2007 20:10:27 GMT, v2)]"
1,"We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n",Ileana Streinu and Louis Theran,"[[Streinu, Ileana, ], [Theran, Louis, ]]",math.CO cs.CG,To appear in Graphs and Combinatorics,None,0704.0002,None,http://arxiv.org/licenses/nonexclusive-distrib/1.0/,None,Louis Theran,Sparsity-certifying Graph Decompositions,2008-12-13,"[(Sat, 31 Mar 2007 02:26:18 GMT, v1), (Sat, 13 Dec 2008 17:26:00 GMT, v2)]"
2,"The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n",Hongjun Pan,"[[Pan, Hongjun, ]]",physics.gen-ph,"23 pages, 3 figures",None,0704.0003,None,None,None,Hongjun Pan,The evolution of the Earth-Moon system based on the dark matter field\n fluid model,2008-01-13,"[(Sun, 1 Apr 2007 20:46:54 GMT, v1), (Sat, 8 Dec 2007 23:47:24 GMT, v2), (Sun, 13 Jan 2008 00:36:28 GMT, v3)]"
3,We show that a determinant of Stirling cy

In [ ]:
clean_text = udf(lambda x: text_prepare(x))
df_pro=df.selectExpr("id","title","abstract").\
            withColumn("origin",lit("arXiv")).\
            withColumn("abstract", clean_text(df.abstract)).\
            withColumn("abstract", col("abstract").alias("", metadata={"maxlength":2048}))
        
        

df.select("id",
          "title",
          "authors",
          "categories",
          "doi",
          "comments",
          "journal-ref",
          "license",
          "report-no",
          "submitter",
          "update_date")



In [ ]:
df_papers_nips.select('id','title')

In [ ]:
df_author=df.selectExpr("id","explode(authors_parsed) as e")
df_author=df_author.withColumn("author",concat_ws(" ",col("e")))
df_author=df_author.select("id","author")

In [ ]:
df_final=df_pro.select("id","title","author",)

df_final.show(10)

In [47]:
df_abs=df.select("abstract")

In [57]:
fix_col_var(df,"abstract").show(10)

+--------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+
|abstract|             authors|      authors_parsed|       categories|            comments|                 doi|       id|         journal-ref|             license|           report-no|         submitter|               title|update_date|            versions|
+--------+--------------------+--------------------+-----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+
|    null|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|           hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                null|    ANL-HEP-PR-07-12|    Pavel Nadolsky|Calculation of pr..

In [59]:
df.columns

['abstract',
 'authors',
 'authors_parsed',
 'categories',
 'comments',
 'doi',
 'id',
 'journal-ref',
 'license',
 'report-no',
 'submitter',
 'title',
 'update_date',
 'versions']